In [ ]:
from geoalchemy2 import Geometry
from shapely.geometry import MultiPolygon
from sqlalchemy.orm import sessionmaker
from sqlalchemy.dialects.sqlite import insert
from datetime import datetime, timedelta
from sqlalchemy.sql import text
import time
import pandas as pd
import sys
sys.path.append('../')

import app.config.mapping as mapping
import app.shared.downloader as downloader
import app.config.connection as connection

In [ ]:
url = "https://an-url-that-you-can-retrive-data-like-json-csv-xml-or-others.com"

engine =connection.getConnectionSqlAlchemyGetEngine()

In [ ]:
result_data = downloader.readZipOnlineFile(url)

In [ ]:
result_data.set_crs('EPSG:4674', allow_override=True)

result_data = result_data.rename(columns=lambda x: x.lower())
result_data = result_data.rename(columns={'geometry': 'column_4'})
result_data['column_4'] = result_data['column_4'].apply(lambda x: MultiPolygon([x]) if x.geom_type=='Polygon' else x)

registry_array = []
limite_data = datetime.today() - timedelta(days=60)
result_data['column_3'] = pd.to_datetime(result_data['column_3'])
filtro = result_data['column_3'] >= limite_data
dados_60_dias = result_data[filtro]

In [ ]:
for index, row in dados_60_dias.iterrows():     

    _registry = {
        'integerColumn ':row['column_1'],
        'stringColumn':row['column_2'],
        'dateColumn': row['column_3'],
        'geomColumn': row['column_4'],
        'floatColumn': row['column_5'],
        'textoColumn': row['column_6'],
        'boolColumn': row['column_7'],
        'date_timeColumn': row['column_8']
    }
    registry_array.append(_registry)

In [ ]:
if(len(registry_array) > 0):
        
    Session = sessionmaker(bind=engine)
    session = Session()
    insert_stmt = insert(mapping.MyTable).values(registry_array)     
    on_conflict_stmt = insert_stmt.on_conflict_do_nothing()
    session.execute(on_conflict_stmt)
    session.commit()     
    print("inserts finalizados")        
else:
    print('nenhum registro adicionado.')